# The VTK Basics

## Basic Classes

The VTK rendering pipeline consists of three main components:
 - `vtkRenderWindow`: Represents the actual window
 - `vtkRenderWindowInteractor`: Handles input events and reacts (mouse interaction etc.)
 - `vtkRenderer`: Renders a collection of `vtkActor`s to an output window

In [1]:
import vtk

In [2]:
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

In [3]:
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renderWindow)

In [4]:
interactor.Initialize()
interactor.Start()

This should give us an interactive render window

Re-running the render window after closing it causes deadlocks.
However, a new render window can use the same renderer.

**However, VTK and PyVTK do not actually work too well with Jupyter Notebooks.**
If you run above cell again after closing the window, the kernel will freeze! You will need to create a new `Interactor` and `RenderWindow` if you want to re-run something!

In [5]:
renderWindow2 = vtk.vtkRenderWindow()
renderWindow2.AddRenderer(renderer)
interactor2 = vtk.vtkRenderWindowInteractor()
interactor2.SetRenderWindow(renderWindow2)
interactor2.Initialize()
interactor2.Start()

So let's refactor this into a method so we can reuse this

In [6]:
def show_renderer(renderer:vtk.vtkRenderer, size=(800, 600)):
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(size)
    interactor = vtk.vtkRenderWindowInteractor()
    interactor.SetRenderWindow(renderWindow)
    interactor.Initialize()
    interactor.Start()

Now we can re-run the method any number of times!

In [7]:
show_renderer(renderer)

The renderer has a list of objects to render.

Renderer objects (vtkProps) can either be actors (e.g. meshes) or volumes.

In [8]:
cubeActor = vtk.vtkActor()
renderer.AddActor(cubeActor)

In [9]:
show_renderer(renderer)

## Rendering Meshes
In order to actually render things, we need two additional things:
 - A `vtkCamera` to set the virtual viewpoint
 - A number of `vtkProp`s which encapsulate the objects to be rendered

In [10]:
Camera = vtk.vtkCamera()
Camera.SetViewUp(0, 0, -1)
Camera.SetPosition(0, -1, 0)
Camera.SetFocalPoint(0.5, 0.5, 0.5)
Camera.ComputeViewPlaneNormal()
Camera.Azimuth(30.0)
Camera.Elevation(30.0)

In [11]:
cubeActor = vtk.vtkActor()

In [12]:
renderer.AddActor(cubeActor)
show_renderer(renderer)

Still nothing! That's because the cubeActor does not have any data yet!

To create data to render, we need to define:

- vertex table
- face table (defining triangles by connecting vertices)

In [13]:
def makeVtkIdList(it)->vtk.vtkIdList:
    vil = vtk.vtkIdList()
    for i in it:
        vil.InsertNextId(int(i))
    return vil

def make_cube()->vtk.vtkPolyData:
    x = [(0.0, 0.0, 0.0), (1.0, 0.0, 0.0), (1.0, 1.0, 0.0), (0.0, 1.0, 0.0),
         (0.0, 0.0, 1.0), (1.0, 0.0 ,1.0), (1.0, 1.0, 1.0), (0.0, 1.0, 1.0)]

    # pts = array of 6 4-tuples of vtkIdType (int) representing the faces
    #     of the cube in terms of the above vertices
    faces = [(0,1,2,3), (4,5,6,7), (0,1,5,4),
           (1,2,6,5), (2,3,7,6), (3,0,4,7)]

    # We'll create the building blocks of polydata including data attributes.
    # Load the point, cell, and data attributes.
    points = vtk.vtkPoints()
    for i,p in enumerate(x):       points.InsertPoint(i, p)
    
    polys = vtk.vtkCellArray()
    for f in faces: polys.InsertNextCell( makeVtkIdList(f) )

    # We now assign the pieces to the vtkPolyData.
    cube = vtk.vtkPolyData()
    cube.SetPoints(points)
    cube.SetPolys(polys)
    return cube

Allright so let's create the cube data!

In [14]:
cube = make_cube()

The poly data is just the abstract representation of the data on the CPU.

To actually associate the data with the actor, it needs to be *mapped* - this means that the data is mapped to the GPU memory. 

`vtkPolyDataMapper` takes care of that for polygonal data types (meshes):

In [15]:
cubeMapper = vtk.vtkPolyDataMapper()
cubeMapper.SetInputData(cube)

Now we can associate the data with the actor by setting the mapper!

In [16]:
cubeActor.SetMapper(cubeMapper)

While we're at it set a background color

In [17]:
renderer.SetBackground(vtk.vtkNamedColors().GetColor3d('White')) 

Finally, we can see our wonderful cube!

In [18]:
show_renderer(renderer)

## This is going to be tedious.. or is it??

In [19]:
import pyvista as pv
import numpy as np
from pyvistaqt import BackgroundPlotter

In [24]:
wnd = BackgroundPlotter()

See https://docs.pyvista.org/examples/00-load/create-poly.html

In [21]:
# mesh points
vertices = np.array([[0, 0, 0],
                     [1, 0, 0],
                     [1, 1, 0],
                     [0, 1, 0],
                     [0.5, 0.5, -1]])

# mesh faces
faces = np.hstack([[4, 0, 1, 2, 3],  # square
                   [3, 0, 1, 4],     # triangle
                   [3, 1, 2, 4]])    # triangle

surf = pv.PolyData(vertices, faces)

In [25]:
wnd.add_mesh(surf)

(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f77eb359dc0

In [26]:
wnd.add_actor(cubeActor)

((vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f77e5ba5d00,
 (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLProperty)0x7f77e919ca00)

## Let's go for something volumetric!

In [24]:
vol_data,_,_ = np.meshgrid(np.linspace(0.0, 0.7, 128), np.linspace(0.0, 1.0, 128), np.linspace(0.0, 1.0, 128))
vol_data = vol_data ** 2

In [25]:
vol_data[30:50,40:60,30:50] = 0.5 # a cube
vol_data[30:50,20:80,75:85] = 0.3 # another cube
vol_data[20:30,50:60,60:70] = 0.2

In [26]:
vol_actor = wnd.add_volume(vol_data, clim=(0.1, 0.6))